In [ ]:
# import necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az



WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


In [3]:

# Load your data
df = pd.read_csv("../data/processed/brent_clean.csv", parse_dates=["Date"])
df.set_index("Date", inplace=True)

# Compute log returns
df["log_return"] = np.log(df["Price"]).diff()
log_returns = df["log_return"].dropna()

## PyMC Model with One Changepoint

In [ ]:
# Bayesian Changepoint Detection
with pm.Model() as changepoint_model:
    # Prior for changepoint location (uniform over time indices)
    cp_index = pm.DiscreteUniform("cp_index", lower=0, upper=len(log_returns) - 1)
    
    # Mean and sigma before changepoint
    mu_1 = pm.Normal("mu_1", mu=0, sigma=0.05)
    sigma_1 = pm.HalfNormal("sigma_1", sigma=0.05)

    # Mean and sigma after changepoint
    mu_2 = pm.Normal("mu_2", mu=0, sigma=0.05)
    sigma_2 = pm.HalfNormal("sigma_2", sigma=0.05)

    # Time index for data
    idx = np.arange(len(log_returns))

    # Define which mean/sigma applies based on changepoint
    mu = pm.math.switch(idx <= cp_index, mu_1, mu_2)
    sigma = pm.math.switch(idx <= cp_index, sigma_1, sigma_2)

    # Likelihood
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=log_returns.values)

    # Inference
    trace = pm.sample(2000, tune=1000, target_accept=0.95, return_inferencedata=True)


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [cp_index]
>NUTS: [mu_1, sigma_1, mu_2, sigma_2]


Output()

## Posterior Plot & Detected Changepoint

In [ ]:
# Convert changepoint index to date
cp_samples = trace.posterior["cp_index"].values.flatten()
cp_mean_idx = int(np.mean(cp_samples))
cp_date = log_returns.index[cp_mean_idx]

# Plot log returns with changepoint
plt.figure(figsize=(15, 5))
plt.plot(log_returns.index, log_returns, label="Log Returns")
plt.axvline(cp_date, color='red', linestyle='--', label=f"Changepoint ~ {cp_date.date()}")
plt.title("Bayesian Changepoint Detection")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Summarize Posterior

In [ ]:
az.plot_posterior(trace, var_names=["cp_index", "mu_1", "mu_2", "sigma_1", "sigma_2"])
